In [10]:
import sys
from pathlib import Path
import shutil
import json
import csv

import requests

DIR = Path('..')
sys.path.append(str(DIR))

import wavetrace as wt

DATA_DIR = DIR/'data'
IN_DIR = DATA_DIR/'sample_inputs'
OUT_DIR = DATA_DIR/'sample_outputs'

%load_ext autoreload
%autoreload 2



The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [14]:
# Get SRTM tiles covering NZ 

# NZ sample points:
path = DATA_DIR/'nz_placenames_2008.csv' 
lonlats = []
with path.open() as src:
    reader = csv.DictReader(src)
    for row in reader:
        lonlats.append([float(row['GEOMETRY_X']), float(row['GEOMETRY_Y'])])
nz_tids = wt.get_srtm_tile_ids(lonlats)
print(len(nz_tids), nz_tids)
    
# Make polygons and then visually check them on https://geojson.io
polygons = wt.get_polygons(nz_tids)
collection = {'type': 'FeatureCollection', 'features': polygons}
path = DIR/'data'/'nz_srtm_polygons.geojson'
with path.open('w') as tgt:
    json.dump(collection, tgt)

66 {'S36E173', 'S36E174', 'S44E173', 'S46E170', 'S48E167', 'S37E175', 'S40E175', 'S42E175', 'S38E175', 'S39E174', 'S38E177', 'S47E169', 'S43E173', 'S44E170', 'S46E166', 'S38E174', 'S42E172', 'S41E172', 'S37E176', 'S45E170', 'S44E169', 'S46E169', 'S40E177', 'S40E174', 'S38E178', 'S41E173', 'S42E174', 'S39E176', 'S39E178', 'S47E166', 'S47E170', 'S46E171', 'S45E172', 'S42E173', 'S40E173', 'S41E174', 'S40E178', 'S45E171', 'S38E176', 'S41E176', 'S47E167', 'S43E170', 'S47E168', 'S42E176', 'S44E168', 'S43E172', 'S46E167', 'S45E169', 'S45E167', 'S36E175', 'S37E174', 'S43E171', 'S35E172', 'S45E168', 'S42E171', 'S35E173', 'S39E177', 'S41E175', 'S48E168', 'S37E173', 'S44E172', 'S46E168', 'S40E176', 'S39E175', 'S43E174', 'S44E171'}


In [20]:
path = Path('/home/araichev/srtmnz/srtm1')
url = "http://e4ftl01.cr.usgs.gov/SRTM/SRTMGL1.003/2000.02.11/"
for i, tid in enumerate(sorted(nz_tids)):
    filename = tid + '.SRTMGL1.hgt.zip'
    href = url + filename
    print(i, href)


0 http://e4ftl01.cr.usgs.gov/SRTM/SRTMGL1.003/2000.02.11/S35E172.SRTMGL1.hgt.zip
1 http://e4ftl01.cr.usgs.gov/SRTM/SRTMGL1.003/2000.02.11/S35E173.SRTMGL1.hgt.zip
2 http://e4ftl01.cr.usgs.gov/SRTM/SRTMGL1.003/2000.02.11/S36E173.SRTMGL1.hgt.zip
3 http://e4ftl01.cr.usgs.gov/SRTM/SRTMGL1.003/2000.02.11/S36E174.SRTMGL1.hgt.zip
4 http://e4ftl01.cr.usgs.gov/SRTM/SRTMGL1.003/2000.02.11/S36E175.SRTMGL1.hgt.zip
5 http://e4ftl01.cr.usgs.gov/SRTM/SRTMGL1.003/2000.02.11/S37E173.SRTMGL1.hgt.zip
6 http://e4ftl01.cr.usgs.gov/SRTM/SRTMGL1.003/2000.02.11/S37E174.SRTMGL1.hgt.zip
7 http://e4ftl01.cr.usgs.gov/SRTM/SRTMGL1.003/2000.02.11/S37E175.SRTMGL1.hgt.zip
8 http://e4ftl01.cr.usgs.gov/SRTM/SRTMGL1.003/2000.02.11/S37E176.SRTMGL1.hgt.zip
9 http://e4ftl01.cr.usgs.gov/SRTM/SRTMGL1.003/2000.02.11/S38E174.SRTMGL1.hgt.zip
10 http://e4ftl01.cr.usgs.gov/SRTM/SRTMGL1.003/2000.02.11/S38E175.SRTMGL1.hgt.zip
11 http://e4ftl01.cr.usgs.gov/SRTM/SRTMGL1.003/2000.02.11/S38E176.SRTMGL1.hgt.zip
12 http://e4ftl01.cr.usgs.

In [ ]:
# def parse_raster_query(r):
#     layers = r['rasterQuery']['layers']
#     for layer_id, d in layers.items():
#         if d['status'] == 'ok':
#             return layer_id

# def get_nzsos_tile_name(lon, lat):
#     for i, j in [(0, 10), (10, 20), (20, 30)]:
#         url = "http://api.koordinates.com/api/rasterQuery.json/"
#         params = {
#             'key': wt.get_secret('KOORDINATES_API_KEY'),
#             'layer': wt.NZSOSDEM[i:j],
#             'x': lon, 
#             'y': lat,
#         }
#         r = requests.get(url, params=params).json()
#         if r.status_code != requests.codes.ok:
#             raise ValueError('Failed to query', url)
#         layer_id = parse_raster_query(r)
#         if layer_id is not None:
#             break

#     return layer_id


In [ ]:
# # Download elevation test data

# TEST_DATA_DIR = DIR/'tests'/'data'
# ts = wt.read_transmitters(TEST_DATA_DIR/'transmitters.csv')[:2]
# lon_lats = wt.get_lon_lats(ts)
# tiles = wt.get_srtm_tile_names(lon_lats)
# print('Tiles to download =', tiles)
    
# wt.download_srtm_nasa(tiles, TEST_DATA_DIR)

In [ ]:
transmitters = wt.read_transmitters(IN_DIR/'transmitters.csv')[:2]
high_definition = False
print(transmitters)

In [ ]:
# Download topography data

lon_lats = wt.get_lon_lats(transmitters)
tiles = wt.get_srtm_tile_names(lon_lats)
print('Tiles to download =', tiles)

username = None
password = None
if high_definition:
    path = IN_DIR/'srtm1'
    try:
        username = wt.get_secret('NASA_USERNAME')
        password = wt.get_secret('NASA_PASSWORD')
    except:
        pass
else:
    path = IN_DIR/'srtm3'
    
wt.download_srtm_nasa(tiles, path, high_definition=high_definition,
  username=username, password=password)

In [ ]:
# Create SPLAT transmitter files

in_path = IN_DIR/'transmitters.csv'
out_path = OUT_DIR/'splat_files'
if out_path.exists():
    shutil.rmtree(str(out_path))
    
wt.create_splat_transmitter_files(in_path, out_path)

% ls {out_path}

In [ ]:
# Create SPLAT topography files

if high_definition:
    in_path = IN_DIR/'srtm1'
else:
    in_path = IN_DIR/'srtm3'
out_path = OUT_DIR/'splat_files'

wt.create_splat_topography_files(in_path, out_path, high_definition=high_definition)

% ls {out_path}

In [ ]:
# Create coverage reports

transmitter_names = [t['name'] for t in transmitters[:1]]
in_path = OUT_DIR/'splat_files'
out_path = OUT_DIR/'coverage_reports'
wt.create_coverage_reports(in_path, out_path, transmitter_names, 
  high_definition=high_definition)

% ls {out_path}

In [ ]:
# Post-process coverage reports

path = OUT_DIR/'coverage_reports'
wt.postprocess_coverage_reports(path, delete_ppm=False)

% ls {path}